In [2]:
!pip3 install pyrealsense2 open3d

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 15.6 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 23.8 MB/s eta 0:00:00 MB/s eta 0:00:0101mm
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.5/746.5 kB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 87.2 MB/s eta 0:00:000m eta 0:00:010:0101
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.6/306.6 kB 36.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [5]:
import pyrealsense2 as rs
import numpy as np
import cv2
import open3d as o3d

# Initialize RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()

# Enable depth and color streams
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)

# Get depth scale for converting depth values to meters
profile = pipeline.get_active_profile()
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()

# Intrinsic parameters
intrinsics = profile.get_stream(rs.stream.depth).as_video_stream_profile().get_intrinsics()
fx, fy, cx, cy = intrinsics.fx, intrinsics.fy, intrinsics.ppx, intrinsics.ppy

def depth_to_3d(x, y, depth):
    """ Convert pixel (x, y) and depth to 3D world coordinates. """
    z = depth * depth_scale
    if z == 0:  # Ignore invalid depth points
        return None
    X = (x - cx) * z / fx
    Y = (y - cy) * z / fy
    return np.array([X, Y, z])

def detect_ball(color_image):
    """ Detect a ball in the color image using color thresholding. """
    hsv = cv2.cvtColor(color_image, cv2.COLOR_BGR2HSV)
    
    # Define color range for the ball (adjust as needed)
    lower_red = np.array([0, 120, 70])
    upper_red = np.array([10, 255, 255])
    
    mask = cv2.inRange(hsv, lower_red, upper_red)

    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        (x, y), radius = cv2.minEnclosingCircle(largest_contour)
        if radius > 5:  # Filter out small noise
            return int(x), int(y), int(radius)
    return None

try:
    while True:
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()

        if not depth_frame or not color_frame:
            continue

        # Convert frames to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        # Detect the ball
        ball = detect_ball(color_image)
        if ball:
            x, y, radius = ball

            # Get depth value at the detected position
            depth_value = depth_image[y, x]
            ball_position = depth_to_3d(x, y, depth_value)

            if ball_position is not None:
                print(f"Ball detected at: {ball_position}")

                # Draw the detected ball on the color image
                cv2.circle(color_image, (x, y), radius, (0, 255, 0), 2)
                cv2.putText(color_image, f"3D: {ball_position[0]:.2f}, {ball_position[1]:.2f}, {ball_position[2]:.2f}", 
                            (x + 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Show images
        cv2.imshow("Color Image", color_image)
        cv2.imshow("Depth Image", cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET))

        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    pipeline.stop()
    cv2.destroyAllWindows()


Ball detected at: [ 0.22144086 -0.07245801  0.28800001]
Ball detected at: [ 0.2058887  -0.05884855  0.36000002]
Ball detected at: [ 0.18596455 -0.02145088  0.45800002]
Ball detected at: [0.18307854 0.18137748 0.89800004]
Ball detected at: [0.14862572 0.24328    0.93000004]
Ball detected at: [0.20428502 0.21872994 0.88900004]
Ball detected at: [0.27563504 0.19599198 0.86000004]
Ball detected at: [0.3277388  0.19486402 0.79200004]
Ball detected at: [0.42269727 0.22388452 0.79300004]
Ball detected at: [0.21707881 1.57690388 3.02800014]
Ball detected at: [0.58467819 0.09260014 1.23500006]
Ball detected at: [ 1.50972239 -0.9009593   2.0320001 ]
Ball detected at: [-0.24425997 -0.20157746  0.37200002]
Ball detected at: [-0.20638047 -0.21875405  0.37500002]
Ball detected at: [-0.17145095 -0.22272884  0.35500002]
Ball detected at: [-0.01669573 -0.24877912  0.62300003]
Ball detected at: [-0.02293157 -0.24958149  0.61700003]
Ball detected at: [-0.01960532 -0.25431861  0.61300003]
Ball detected at